## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps



In [2]:
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,-21.2078,-159.7750,84.25,58,75,5.75,broken clouds
1,1,Albany,US,42.6001,-73.9662,55.63,65,17,1.99,few clouds
2,2,Vaini,TO,-21.2000,-175.2000,78.96,65,20,9.22,few clouds
3,3,Hobart,AU,-42.8794,147.3294,53.02,77,75,9.22,broken clouds
4,4,Kamien Pomorski,PL,53.9685,14.7726,44.96,66,93,12.41,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,-21.2078,-159.7750,84.25,58,75,5.75,broken clouds
2,2,Vaini,TO,-21.2000,-175.2000,78.96,65,20,9.22,few clouds
11,11,Rikitea,PF,-23.1203,-134.9692,76.95,74,65,21.85,broken clouds
13,13,Ugoofaaru,MV,5.6667,73.0000,83.10,77,100,22.17,overcast clouds
15,15,Zhuhai,CN,22.2769,113.5678,79.86,60,0,9.22,clear sky
16,16,Atuona,PF,-9.8000,-139.0333,79.09,70,0,14.61,clear sky
19,19,Cairns,AU,-16.9167,145.7667,86.11,68,1,6.91,clear sky
23,23,The Valley,AI,18.2170,-63.0578,80.82,86,100,16.42,overcast clouds
32,32,Lorengau,PG,-2.0226,147.2712,85.86,66,8,9.33,clear sky
33,33,Hithadhoo,MV,-0.6000,73.0833,84.72,69,100,17.22,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             207
City                   207
Country                207
Lat                    207
Lng                    207
Max Temp               207
Humidity               207
Cloudiness             207
Wind Speed             207
Current Description    207
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

Unnamed: 0             207
City                   207
Country                207
Lat                    207
Lng                    207
Max Temp               207
Humidity               207
Cloudiness             207
Wind Speed             207
Current Description    207
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Avarua,CK,84.25,broken clouds,-21.2078,-159.7750,
2,Vaini,TO,78.96,few clouds,-21.2000,-175.2000,
11,Rikitea,PF,76.95,broken clouds,-23.1203,-134.9692,
13,Ugoofaaru,MV,83.10,overcast clouds,5.6667,73.0000,
15,Zhuhai,CN,79.86,clear sky,22.2769,113.5678,
16,Atuona,PF,79.09,clear sky,-9.8000,-139.0333,
19,Cairns,AU,86.11,clear sky,-16.9167,145.7667,
23,The Valley,AI,80.82,overcast clouds,18.2170,-63.0578,
32,Lorengau,PG,85.86,clear sky,-2.0226,147.2712,
33,Hithadhoo,MV,84.72,overcast clouds,-0.6000,73.0833,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [11]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Avarua,CK,84.25,broken clouds,-21.2078,-159.7750,Paradise Inn
2,Vaini,TO,78.96,few clouds,-21.2000,-175.2000,Keleti Beach Resort
11,Rikitea,PF,76.95,broken clouds,-23.1203,-134.9692,People ThankYou
13,Ugoofaaru,MV,83.10,overcast clouds,5.6667,73.0000,Ungoofaaru Inn
15,Zhuhai,CN,79.86,clear sky,22.2769,113.5678,珠海粤财假日酒店
...,...,...,...,...,...,...,...
681,Honiara,SB,87.76,clear sky,-9.4333,159.9500,Heritage Park Hotel
686,Waingapu,ID,84.70,scattered clouds,-9.6567,120.2641,Pondok Wisata Elim
687,Biloela,AU,75.65,scattered clouds,-24.4167,150.5000,Silo Motor Inn Biloela
688,Puerto Vallarta,MX,78.71,clear sky,20.6204,-105.2307,Sunset Plaza Beach Resort & Spa


In [12]:
nan_value = float("NaN")
hotel_df = hotel_df.replace("", nan_value)

In [13]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Avarua,CK,84.25,broken clouds,-21.2078,-159.7750,Paradise Inn
2,Vaini,TO,78.96,few clouds,-21.2000,-175.2000,Keleti Beach Resort
11,Rikitea,PF,76.95,broken clouds,-23.1203,-134.9692,People ThankYou
13,Ugoofaaru,MV,83.10,overcast clouds,5.6667,73.0000,Ungoofaaru Inn
15,Zhuhai,CN,79.86,clear sky,22.2769,113.5678,珠海粤财假日酒店
...,...,...,...,...,...,...,...
681,Honiara,SB,87.76,clear sky,-9.4333,159.9500,Heritage Park Hotel
686,Waingapu,ID,84.70,scattered clouds,-9.6567,120.2641,Pondok Wisata Elim
687,Biloela,AU,75.65,scattered clouds,-24.4167,150.5000,Silo Motor Inn Biloela
688,Puerto Vallarta,MX,78.71,clear sky,20.6204,-105.2307,Sunset Plaza Beach Resort & Spa


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))